# Data Engineer Exercise


## Description

In the following code, I will use the three files "cons.csv", "cons_email.csv", and "cons_email_chapter_subscriptions.csv" to create two new files: "people" and "acquisition_facts" based on the required metrics. 

## Packages and Data Importing

In [1]:
# Packages: pandas
import pandas as pd

In [31]:
# Import .csv files from working directory 

cons = pd.read_csv("cons.csv")
cons_email = pd.read_csv("cons_email.csv")
cons_email_chapter_subscription = pd.read_csv(
    "cons_email_chapter_subscription.csv")

In [3]:
# Check formatting of data 

print(cons.columns)
print(cons_email.columns)
print(cons_email_chapter_subscription.columns)

Index(['cons_id', 'prefix', 'firstname', 'middlename', 'lastname', 'suffix',
       'salutation', 'gender', 'birth_dt', 'title', 'employer', 'occupation',
       'income', 'source', 'subsource', 'userid', 'password', 'is_validated',
       'is_banned', 'change_password_next_login', 'consent_type_id',
       'create_dt', 'create_app', 'create_user', 'modified_dt', 'modified_app',
       'modified_user', 'status', 'note'],
      dtype='object')
Index(['cons_email_id', 'cons_id', 'cons_email_type_id', 'is_primary', 'email',
       'canonical_local_part', 'domain', 'double_validation', 'create_dt',
       'create_app', 'create_user', 'modified_dt', 'modified_app',
       'modified_user', 'status', 'note'],
      dtype='object')
Index(['cons_email_chapter_subscription_id', 'cons_email_id', 'chapter_id',
       'isunsub', 'unsub_dt', 'modified_dt'],
      dtype='object')


## Task 1: "people" file

In [33]:
# We want "email" from the cons_email.csv file
# We want "create_dt" and "modified_dt" and "source" from the cons.csv file
# Finally, we want "is_unsub" from the cons_email_chapter_subscription.csv file
# If a primary email from "email" is not in cons_email_chapter_subscription, it
# receives a 0 for is_unsub, because it is still subscribed. 

# Merge data frames - outer join first to not lose any data 
cons_join = cons.merge(cons_email, on = ["cons_id"], how = "outer")
cons_join = cons_join.merge(cons_email_chapter_subscription, 
                            on = ["cons_email_id"], how = "outer")

In [34]:
# Keep only primary emails 

cons_join = cons_join[cons_join["is_primary"] == 1.0]

In [35]:
# Keep only rows where chapter_id is 1

cons_join = cons_join[cons_join["chapter_id"] == 1.0]

In [36]:
# As indicated in the instructions, if an email has a chapter_id = 1, but does
# not have isunsub listing, then isunsub should get 0. 
# It looks like this condition does not occur. 

cons_join["isunsub"].isnull().values.any()

False

In [37]:
# Create people dataframe 
# The create_t and modified_dt we want are the "x" ones

people = cons_join[["email", "source", "isunsub", "create_dt_x", 
                    "modified_dt_x"]].reset_index(drop = True)

# Rename columns to requested

people.columns = ["email", "code", "is_unsub", "created_dt", "updated_dt"]

In [38]:
# Check if new df was created properly

people.head(10)

,email,code,is_unsub,created_dt,updated_dt
0,caustin@spears-carson.com,facebook,1.0,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57"
1,klewis@ford.biz,NaN,1.0,"Fri, 2008-08-22 19:20:28","Fri, 2020-06-05 18:13:57"
2,stephenhamilton@gmail.com,NaN,1.0,"Thu, 2007-07-19 18:28:09","Tue, 1971-04-27 06:53:53"
3,norr@donovan.com,twitter,1.0,"Thu, 2004-11-11 02:32:27","Mon, 2018-11-05 17:26:30"
4,christy72@yahoo.com,facebook,1.0,"Thu, 1984-03-29 19:14:34","Fri, 1993-12-17 12:10:30"
5,hernandeztammy@hotmail.com,google,0.0,"Fri, 2016-07-29 09:28:21","Sat, 2014-01-11 11:57:25"
6,zward@gonzalez.com,NaN,1.0,"Mon, 1995-08-21 12:52:43","Mon, 1998-09-07 22:21:38"
7,nroberson@yahoo.com,twitter,1.0,"Sun, 1990-12-23 12:31:26","Tue, 2008-04-29 10:23:38"
8,kellyspears@gmail.com,facebook,0.0,"Thu, 2005-11-17 16:05:17","Sat, 2013-03-23 17:57:34"
9,rsilva@marshall-bernard.com,organic,1.0,"Sat, 1980-02-09 12:57:33","Tue, 1986-10-07 18:03:07"


In [39]:
# Check on datatypes of new df 

people.dtypes

email          object
code           object
is_unsub      float64
created_dt     object
updated_dt     object
dtype: object

In [40]:
# Change datatypes to requested 

people["email"] = people["email"].astype("string")
people["code"] = people["code"].astype("string")
people["is_unsub"] = people["is_unsub"].astype("bool")
people["created_dt"] = pd.to_datetime(people["created_dt"])
people["updated_dt"] = pd.to_datetime(people["updated_dt"])

In [41]:
# Check datatypes again

people.dtypes

email                 string
code                  string
is_unsub                bool
created_dt    datetime64[ns]
updated_dt    datetime64[ns]
dtype: object

In [42]:
# Save df to working directory 

people.to_csv("people.csv")

## Task 2: "acquisition_facts" file 

In [43]:
# Get dates from datetime column

acquisition_dates = people["created_dt"].dt.date

In [44]:
# Create df

acquisition_facts = pd.value_counts(acquisition_dates).to_frame().reset_index()

In [45]:
# Rename columns

acquisition_facts.columns = ["acquisition_date", "acquisitions"]

In [46]:
# Sort by date 

acquisition_facts = acquisition_facts.sort_values(
    by = "acquisition_date", ascending = False).reset_index(drop = True)

In [47]:
# Check to see if dataframe was created properly 

acquisition_facts.head(10)

,acquisition_date,acquisitions
0,2020-07-01,8
1,2020-06-30,11
2,2020-06-29,11
3,2020-06-28,11
4,2020-06-27,7
5,2020-06-26,11
6,2020-06-25,11
7,2020-06-24,14
8,2020-06-23,12
9,2020-06-22,15


In [48]:
# Check datatypes

acquisition_facts.dtypes

acquisition_date    object
acquisitions         int64
dtype: object

In [49]:
# Change datatypes to match requested
# Change date column to date  

acquisition_facts["acquisition_date"] = pd.to_datetime(
    acquisition_facts["acquisition_date"]).dt.normalize()

In [50]:
# Check datatypes again

acquisition_facts.dtypes

acquisition_date    datetime64[ns]
acquisitions                 int64
dtype: object

In [51]:
# Save to working directory 

acquisition_facts.to_csv("acquisition_facts.csv")